In [ ]:
from src.sampling.images import plot_frame_count_distributions

samples_root_dir = "data/samples"

plot_frame_count_distributions(samples_root_dir)

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11m-cls")
sota.initialize_model()
sota.train_model(optimizer="AdamW", lr0=0.0005)


#metrics = model.val(data="data/img/techniques/val")

In [ ]:
from sota.model import SOTA

sota = SOTA("data", "yolo11n-cls")
sota.initialize_model()
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11m-cls")
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11n-cls")
#sota.train_model()
sota.test_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1")
hpednn.initialize_model()
hpednn.train_model()

In [1]:
from src.hpe_dnn.model import HpeDnn

hpednn_aug = HpeDnn("data", "arch1_augmented")
hpednn_aug.initialize_model()
hpednn_aug.train_model(augment=True)

2025-06-27 14:52:37.514146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751028758.578061   56302 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751028758.871235   56302 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-27 14:52:41.602255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/mnt/c/Projects/climbing-technique-detector/.venv/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWar

loading a fresh model 'arch1_augmented'


NameError: name '_HpeDnn__df_to_dataset' is not defined

In [ ]:
%load_ext tensorboard

from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch2_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH2)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch3_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH3)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_not_norm", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, normalize=False)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_dr_0.3", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, dropout_rate=0.3)
hpednn.train_model()

In [ ]:
%tensorboard --logdir data/runs/hpe_dnn/arch1_balanced/train1/logs/train

In [ ]:
import albumentations as A
from cv2 import imread
import numpy as np

from src.hpe.evaluate import predict_landmarks
from src.hpe.model import build_holistic_model
from src.hpe_dnn.augmentation import __transform_pipeline

image_file_path = "data/img/techniques/train/CROSS_MIDLINE/(Climbing Analysis) Inside Flag - The Move You Never Do But Maybe Should__1647__9.png"
image2_file_path = "data/img/techniques/train/CROSS_MIDLINE/Route1Climb2__972__20.png"

for image_path in [image_file_path, image2_file_path]:

    image = imread(image_path)
    height, width, _ = image.shape
    
    with build_holistic_model() as model:

        result, _ = predict_landmarks(image, model)
        (keypoints, vis, labels) = __to_augmenting_array2(result, height, width)

        vis_keypoints(image, keypoints)   

        transformed = __transform_pipeline(image=image, keypoints=keypoints)
        print(transformed['keypoints'])
        
        vis_keypoints(transformed['image'], transformed['keypoints'])


In [ ]:
from src.hpe.evaluate import __to_np_array

for image_path in [image_file_path, image2_file_path]:

    image = imread(image_path)
    height, width, _ = image.shape
    
    with build_holistic_model() as model:

        result, _ = predict_landmarks(image, model)
        before = __to_np_array(result)
        print(f"Before: {before}")

        (keypoints, vis, labels, mask) = __to_augmenting_array(result, height, width)
        
        #after = __to_feature_vector(keypoints, vis, labels, mask, heigth, width)
        
        transformed = __transform_pipeline(image=image, keypoints=keypoints)
        after = __to_feature_vector(transformed['keypoints'], vis, labels, mask, height, width)
        print(f"After: {after}")

        print(f"Success?: {before == after}")


In [ ]:
print(before.shape)
print(after.shape)
print(labels)
print([label in _used_hand_landmarks for label in labels])
print(type(labels))
print(type(labels[-1]))
print(len(keypoints))
print(len(transformed['keypoints']))

In [ ]:
from src.hpe_dnn.model import read_data

df_path = "data/df/techniques/train.pkl"
train = read_data(df_path)

In [ ]:
from cv2 import imread

from src.hpe.model import build_holistic_model
from src.hpe_dnn.augmentation import __transform_pipeline, __to_augmenting_array, __to_df_row

def augment(series):
    img_path = series["image_path"]
    image = imread(img_path)
    height, width, _ = image.shape
    
    with build_holistic_model() as model:

        keypoints, vis = __to_augmenting_array(series, height, width)
        
        print("Before: " + keypoints)
        
        after = __to_df_row(series, keypoints, vis, height, width)
        
        print(f"After: {after}")


    return series

In [ ]:
from cv2 import imread

from src.hpe_dnn.model import read_data
from src.hpe_dnn.augmentation import __transform_pipeline, __to_augmenting_array, __to_df_row
from src.hpe_dnn.draw import draw_augmented_keypoints

df_path = "data/df/techniques/train.pkl"
train = read_data(df_path)

def print_info(input):
    img_path = input["image_path"]
    image = imread(img_path)
    height, width, _ = image.shape
    
    xyz, vis = __to_augmenting_array(input, height, width)
    
    transformed = __transform_pipeline(image=image, keypoints=xyz)

    draw_augmented_keypoints(transformed['image'], transformed['keypoints'])

    output = __to_df_row(input, transformed['keypoints'], vis, height, width)

    for header in input.index:
        print(f"{header}: {input[header]} ===> {output[header]}")    

    return output

train.head().apply(print_info, axis=1)

In [ ]:
train.head()